#### Son redundantes ciudad y provincia? con cual gano más? 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Imputer


import re

from math import sqrt

In [2]:
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.fecha = pd.to_datetime(df.fecha)

#### incluyo verdaderamente el día del año, estaba mal indexado.

In [5]:
df['year'] = df.fecha.dt.year
df['mes'] = df.fecha.dt.month
df['dia_del_mes'] = df.fecha.dt.day
df['dia_del_anio'] = df.fecha.dt.dayofyear

In [57]:
df.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,year,mes,dia_del_mes,dia_del_anio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2273000.0,2015,8,23,235
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,0.0,0.0,0.0,1.0,1.0,3600000.0,2013,6,28,179
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1200000.0,2015,10,17,290
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,0.0,0.0,0.0,1.0,1.0,650000.0,2012,3,9,69
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1150000.0,2016,6,7,159


In [58]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'year', 'mes', 'dia_del_mes',
       'dia_del_anio'],
      dtype='object')

In [59]:
df.provincia.value_counts()

Distrito Federal         58790
Edo. de México           41607
Jalisco                  21238
Querétaro                16988
Nuevo León               15324
Puebla                   10421
San luis Potosí           8447
Yucatán                   7928
Morelos                   7337
Veracruz                  5762
Quintana Roo              4756
Chihuahua                 4590
Coahuila                  3695
Baja California Norte     3220
Sonora                    2988
Guanajuato                2860
Guerrero                  2678
Hidalgo                   2521
Michoacán                 2471
Tamaulipas                2303
Durango                   2275
Sinaloa                   1806
Aguascalientes            1753
Baja California Sur       1700
Nayarit                   1352
Chiapas                   1126
Colima                    1008
Tabasco                    994
Tlaxcala                   839
Oaxaca                     711
Campeche                   263
Zacatecas                   94
Name: pr

In [60]:
df.ciudad.value_counts()

Querétaro                              12646
Benito Juárez                          11014
Zapopan                                10360
San Luis Potosí                         7925
Mérida                                  7162
Monterrey                               6946
Alvaro Obregón                          6633
Cuauhtémoc                              6614
Naucalpan de Juárez                     6554
Miguel Hidalgo                          5795
Atizapán de Zaragoza                    5783
Tlalpan                                 5721
Huixquilucan                            5718
Coyoacán                                5293
Puebla                                  4636
Guadalajara                             4006
San Andrés Cholula                      3805
Cancún                                  3779
Cuernavaca                              3775
Chihuahua                               3757
Tlalnepantla de Baz                     3588
Cuautitlán Izcalli                      3408
Tlajomulco

In [61]:
conProvincia = ['id','titulo','descripcion','direccion','ciudad','idzona','lat','lng','fecha']

In [62]:
X = df.drop(['precio'] + conProvincia,axis = 1)
y = df['precio']

In [63]:
X = pd.get_dummies(X, dummy_na=False)

In [64]:
X.shape

(240000, 71)

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [16]:
parametros = {'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 1,
 'n_estimators': 100,
 'nthread': 4,
 'random_state': 0,
 'subsample': 1}

In [67]:
modelProv = XGBRegressor(**parametros)

In [68]:
%%time
modelProv.fit(X_train,y_train)

[20:51:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 1min 54s, sys: 6.35 s, total: 2min
Wall time: 30.5 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [69]:
pred = modelProv.predict(X_test)

#### Score teniendo en cuenta solo las provincias, el tipo de propiedad y las variables numericas (incluido numero del año,dia del mes, mes y año)

In [70]:
RMSLE(y_test,pred)

/home/braianvillalba/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


0.36790422181674043

In [72]:
conCiudad = ['id','titulo','descripcion','direccion','provincia','idzona','lat','lng','fecha']

In [73]:
X = df.drop(['precio'] + conCiudad,axis = 1)
y = df['precio']

In [74]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 17 columns):
tipodepropiedad               239954 non-null object
ciudad                        239628 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
gimnasio                      240000 non-null float64
usosmultiples                 240000 non-null float64
piscina                       240000 non-null float64
escuelascercanas              240000 non-null float64
centroscomercialescercanos    240000 non-null float64
year                          240000 non-null int64
mes                           240000 non-null int64
dia_del_mes                   240000 non-null int64
dia_del_anio                  240

In [75]:
X = pd.get_dummies(X, dummy_na=False)

In [76]:
X.shape

(240000, 914)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [80]:
modelCiudad = XGBRegressor(**parametros)

In [81]:
%%time
modelCiudad.fit(X_train,y_train)

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[20:59:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 14min 28s, sys: 13.4 s, total: 14min 41s
Wall time: 3min 49s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [82]:
predC = modelCiudad.predict(X_test)

#### Scores muy similares pero con mucho más tiempo de entrenamiento empleado, por ahora es mejor tener provincia.

In [83]:
RMSLE(y_test,predC)

0.3627428812975696

In [7]:
columnsDrop = ['id','titulo','descripcion','direccion','idzona','lat','lng','fecha']

In [86]:
X = df.drop(['precio'] + columnsDrop,axis = 1)
y = df['precio']

In [87]:
X.shape

(240000, 18)

In [88]:
X = pd.get_dummies(X, dummy_na=False)

##### Ojo con la dimensionalidad!

In [89]:
X.shape

(240000, 946)

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [90]:
modelComplete = XGBRegressor(**parametros)

In [93]:
%%time
modelComplete.fit(X_train,y_train)

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:13:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 13min 57s, sys: 13.4 s, total: 14min 11s
Wall time: 3min 36s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [94]:
resultComplete = modelComplete.predict(X_test)

#### Acá tengo una muy leve mejora pero es muy poca, no vale la pena tomar ciudades (por ahora)

In [95]:
RMSLE(y_test,resultComplete)

0.34858195052841756